In [1]:
import pydra

### Intro to FunctionTask

Task can be created from every function defined by the user by using the `pydra.to_task` decorator.

In [2]:
@pydra.to_task
def add_var(a, b):
    return a + b


Once we define the function and use the decorator, we can create a pydra `Task` with a name and specified input.

In [3]:
task1 = add_var(name="a_plus_b", a=4, b=5)

We can check the type of task1, that should be `FunctionTask`.

In [4]:
type(task1)

pydra.engine.task.FunctionTask

We can also check that the task has correct values of `a` and `b`, that are saved in the task inputs.

In [5]:
print(f"a = {task1.inputs.a}")
print(f"b = {task1.inputs.b}")

a = 4
b = 5


We can also check entire inputs

In [6]:
task1.inputs

Inputs(a=4, b=5, _func=b'\x80\x04\x95u\x01\x00\x00\x00\x00\x00\x00\x8c\x17cloudpickle.cloudpickle\x94\x8c\x0e_fill_function\x94\x93\x94(h\x00\x8c\x0f_make_skel_func\x94\x93\x94h\x00\x8c\r_builtin_type\x94\x93\x94\x8c\x08CodeType\x94\x85\x94R\x94(K\x02K\x00K\x02K\x02KCC\x08|\x00|\x01\x17\x00S\x00\x94N\x85\x94)\x8c\x01a\x94\x8c\x01b\x94\x86\x94\x8c\x1e<ipython-input-2-77d2b7241c56>\x94\x8c\x07add_var\x94K\x01C\x02\x00\x02\x94))t\x94R\x94J\xff\xff\xff\xff}\x94(\x8c\x0b__package__\x94N\x8c\x08__name__\x94\x8c\x08__main__\x94u\x87\x94R\x94}\x94(\x8c\x07globals\x94}\x94\x8c\x08defaults\x94N\x8c\x04dict\x94}\x94\x8c\x0eclosure_values\x94N\x8c\x06module\x94h\x17\x8c\x04name\x94h\x10\x8c\x03doc\x94N\x8c\x0bannotations\x94}\x94\x8c\x08qualname\x94h\x10\x8c\nkwdefaults\x94NutR.')

As you could see, `task.inputs` contains also information about the function, that is an inseparable part of the task.

Once we have task with set input, we can run the task. Since `Task` is callable object, we can simply run:

In [7]:
task1()

Result(output=Output(out=9), runtime=None, errored=False)

As you can see, the result was returned right away, but we can also access it later:

In [8]:
task1.result()

Result(output=Output(out=9), runtime=None, errored=False)

`Result` contains more than just an output, so if we want to get the task output, we can type:

In [9]:
result = task1.result()
result.output.out

9

#### Custom output names
Note that "out" is a default name for the task output. We can always change it:

In [10]:
task1a = add_var(name="a_plus_b", a=4, b=5, output_names="a_plus_b")
task1a()

Result(output=Output(a_plus_b=9), runtime=None, errored=False)

If the function returns two values, we can provide `output_names` as a list:

In [11]:
@pydra.to_task
def modf(a):
    import math
    return math.modf(a)

task2 = modf(name="modf", a=3.5, output_names=["fractional", "integer"])
task2()

Result(output=Output(fractional=0.5, integer=3.0), runtime=None, errored=False)

#### Setting inputs

We don't have to provide the input when we create a task, we can always set it later:

In [12]:
task3 = add_var(name="a_plus_b")
task3.inputs.a = 4
task3.inputs.b = 5
task3()

Result(output=Output(out=9), runtime=None, errored=False)

If we forget to specify the input, `None` will be used as the default value, so the function will return a python error.

In [13]:
task3a = add_var(name="a_plus_b")
task3a.inputs.a = 4
task3a()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

#### Various way of execution

as we mentioned before, `Task` is a callable object, so we can run the task using `__call__` method, but this is not the online way of calling the task. We can use `Submitter` class with a specific plugin to execute the task. Submitter will be explained later, but here is an example of running the task using `ConcurrentFutures`.

In order to use `ConcurrentFutures` in Jupyter notebook, we need to use nest_asyncio and set an additional environmental variable

In [15]:
import nest_asyncio
nest_asyncio.apply()
import os
os.environ["Jupyter"] = "True"
import pydra

Now we can create submitter and run the task using the submitter.

In [16]:
task4 = add_var(name="a_plus_b")
task4.inputs.a = 4
task4.inputs.b = 5

with pydra.Submitter(plugin="cf") as sub:
    sub(task4)
task4.result()

Result(output=Output(out=9), runtime=None, errored=False)

`Task.__call__` method can also use the submitter if we pass it as an argument:

In [17]:
task5 = add_var(name="a_plus_b")
task5.inputs.a = 4
task5.inputs.b = 5

with pydra.Submitter(plugin="cf") as sub:
    task5(submitter=sub)
task5.result()

Result(output=Output(out=9), runtime=None, errored=False)

We could also just provide a name of the plugin and the task will create a submitter for us:

In [18]:
task6 = add_var(name="a_plus_b")
task6.inputs.a = 4
task6.inputs.b = 5

task6(plugin="cf")

task6.result()

Result(output=Output(out=9), runtime=None, errored=False)